In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

Saving flights.parquet to flights.parquet


In [ ]:
import pandas as pd

# Replace 'file_path.parquet' with the path to your Parquet file
df = pd.read_parquet('flights.parquet')

# Display the first few rows of the dataframe
df.head(n=10)

,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,SCHEDULED_DEPARTURE
0,-11.0,-22.0,1448,0.083333
1,-8.0,-9.0,2330,0.166667
2,-2.0,5.0,2296,0.333333
3,-5.0,-9.0,2342,0.333333
4,-1.0,-21.0,1448,0.416667
5,-5.0,8.0,1589,0.416667
6,-6.0,-17.0,1299,0.416667
7,14.0,-10.0,2125,0.500000
8,-11.0,-13.0,1464,0.500000
9,3.0,-15.0,1747,0.500000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 4 columns):
 #   Column               Dtype  
---  ------               -----  
 0   DEPARTURE_DELAY      float64
 1   ARRIVAL_DELAY        float64
 2   DISTANCE             int64  
 3   SCHEDULED_DEPARTURE  float64
dtypes: float64(3), int64(1)
memory usage: 177.6 MB


In [ ]:
df.describe()

,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,SCHEDULED_DEPARTURE
count,5.732926e+06,5.714008e+06,5.819079e+06,5.819079e+06
mean,9.370158e+00,4.407057e+00,8.223565e+02,1.347392e+01
std,3.708094e+01,3.927130e+01,6.077843e+02,4.849804e+00
min,-8.200000e+01,-8.700000e+01,2.100000e+01,1.666667e-02
25%,-5.000000e+00,-1.300000e+01,3.730000e+02,9.283333e+00
50%,-2.000000e+00,-5.000000e+00,6.470000e+02,1.341667e+01
75%,7.000000e+00,8.000000e+00,1.062000e+03,1.750000e+01
max,1.988000e+03,1.971000e+03,4.983000e+03,2.398333e+01


In [ ]:
distance_threshold = 1650
start_time = 1.0
end_time = 20.0

In [ ]:
# Filter the dataset and drop NaN values
filtered_df = df[(df['DISTANCE'] < distance_threshold) &
                 (df['SCHEDULED_DEPARTURE'] >= start_time) &
                 (df['SCHEDULED_DEPARTURE'] < end_time)]

# Calculate the Pearson correlation coefficient between DEPARTURE_DELAY and ARRIVAL_DELAY
correlation = filtered_df[['DEPARTURE_DELAY', 'ARRIVAL_DELAY']].corr(method='pearson')
print(correlation)

# Extract the correlation value
pearson_corr = correlation.loc['DEPARTURE_DELAY', 'ARRIVAL_DELAY']

# Display the correlation value
print(f"Pearson correlation between DEPARTURE_DELAY and ARRIVAL_DELAY for the filtered flights (with NaN handling): {pearson_corr}")

                 DEPARTURE_DELAY  ARRIVAL_DELAY
DEPARTURE_DELAY         1.000000       0.947718
ARRIVAL_DELAY           0.947718       1.000000
Pearson correlation between DEPARTURE_DELAY and ARRIVAL_DELAY for the filtered flights (with NaN handling): 0.9477176515482378


In [ ]:
distance_threshold = 450
start_time = 8.0
end_time = 23.0

In [ ]:
# Filter the dataset and drop NaN values
filtered_df = df[(df['DISTANCE'] < distance_threshold) &
                 (df['SCHEDULED_DEPARTURE'] >= start_time) &
                 (df['SCHEDULED_DEPARTURE'] < end_time)].dropna(subset=['DEPARTURE_DELAY', 'ARRIVAL_DELAY'])

# Calculate the Pearson correlation coefficient between DEPARTURE_DELAY and ARRIVAL_DELAY
correlation = filtered_df[['DEPARTURE_DELAY', 'ARRIVAL_DELAY']].corr(method='pearson')
print(correlation)

# Extract the correlation value
pearson_corr = correlation.loc['DEPARTURE_DELAY', 'ARRIVAL_DELAY']

# Display the correlation value
print(f"Pearson correlation between DEPARTURE_DELAY and ARRIVAL_DELAY for the filtered flights (with NaN handling): {pearson_corr}")

                 DEPARTURE_DELAY  ARRIVAL_DELAY
DEPARTURE_DELAY         1.000000       0.955344
ARRIVAL_DELAY           0.955344       1.000000
Pearson correlation between DEPARTURE_DELAY and ARRIVAL_DELAY for the filtered flights (with NaN handling): 0.955344145831876


1.   Filter for flights traveling less than 2900 miles with scheduled departure starting from 20:00 and before 21:00.
2.   Find the IQR - the difference between Q1 (first quartile) and Q3 (third quartile) - of the arrival delays.
3.   Calculate the lower and upper bounds. The lower bound is Q1 - 1.5 * IQR and the upper bound is Q3 + 1.5 * IQR.
4.   Count the number of arrival delays that are outside the lower and upper bounds. (Arrival delays exactly at the bounds are not outliers.)

In [ ]:
filtered_df = df[(df['DISTANCE'] < 2900) &
                 (df['SCHEDULED_DEPARTURE'] >= 20.0) &
                 (df['SCHEDULED_DEPARTURE'] < 21.0)]


In [ ]:
q1=filtered_df['ARRIVAL_DELAY'].quantile(0.25)
q3=filtered_df['ARRIVAL_DELAY'].quantile(0.75)
IQR=q3-q1
lower_bound=q1-1.5*IQR
upper_bound=q3+1.5*IQR

In [ ]:
print(lower_bound)
print(upper_bound)

-54.0
58.0


In [ ]:
filtered_df[(filtered_df['ARRIVAL_DELAY']<lower_bound) | (filtered_df['ARRIVAL_DELAY']>upper_bound)]

,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,SCHEDULED_DEPARTURE
12503,21.0,63.0,888,20.000000
12509,68.0,104.0,794,20.000000
12531,63.0,87.0,420,20.000000
12567,87.0,110.0,641,20.033333
12579,69.0,78.0,472,20.083333
...,...,...,...,...
5818441,72.0,72.0,925,20.583333
5818464,74.0,67.0,972,20.666667
5818498,541.0,526.0,2454,20.833333
5818514,106.0,97.0,667,20.850000


In [ ]:
import pandas as pd

# Assuming the DataFrame df has already been created and contains the flights data

# Define the criteria for filtering
distance_threshold = 2900
start_time = 20.0
end_time = 21.0

# Filter the dataset based on the criteria
filtered_df = df[(df['DISTANCE'] < distance_threshold) &
                 (df['SCHEDULED_DEPARTURE'] >= start_time) &
                 (df['SCHEDULED_DEPARTURE'] < end_time)]

# Calculate the first quartile (Q1) and third quartile (Q3) of ARRIVAL_DELAY
Q1 = filtered_df['ARRIVAL_DELAY'].quantile(0.25)
Q3 = filtered_df['ARRIVAL_DELAY'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Calculate the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Count the number of arrival delays that are outside the lower and upper bounds
outliers = filtered_df[(filtered_df['ARRIVAL_DELAY'] < lower_bound) |
                       (filtered_df['ARRIVAL_DELAY'] > upper_bound)]

outlier_count = outliers.shape[0]

# Display the results
print(f"IQR of ARRIVAL_DELAY: {IQR}")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")
print(f"Number of arrival delays outside the bounds: {outlier_count}")


IQR of ARRIVAL_DELAY: 28.0
Lower bound: -54.0
Upper bound: 58.0
Number of arrival delays outside the bounds: 22789


In [ ]:
import pandas as pd

# Assuming the DataFrame df has already been created and contains the flights data

# Define the criteria for filtering
distance_threshold = 2200
start_time = 8.0
end_time = 9.0

# Filter the dataset based on the criteria
filtered_df = df[(df['DISTANCE'] < distance_threshold) &
                 (df['SCHEDULED_DEPARTURE'] >= start_time) &
                 (df['SCHEDULED_DEPARTURE'] < end_time)]

# Calculate the first quartile (Q1) and third quartile (Q3) of ARRIVAL_DELAY
Q1 = filtered_df['ARRIVAL_DELAY'].quantile(0.25)
Q3 = filtered_df['ARRIVAL_DELAY'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Calculate the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Count the number of arrival delays that are outside the lower and upper bounds
outliers = filtered_df[(filtered_df['ARRIVAL_DELAY'] < lower_bound) |
                       (filtered_df['ARRIVAL_DELAY'] > upper_bound)]

outlier_count = outliers.shape[0]

# Display the results
print(f"IQR of ARRIVAL_DELAY: {IQR}")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")
print(f"Number of arrival delays outside the bounds: {outlier_count}")


IQR of ARRIVAL_DELAY: 17.0
Lower bound: -39.5
Upper bound: 28.5
Number of arrival delays outside the bounds: 27269


In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

Saving house-rent.csv to house-rent (1).csv


In [ ]:
import pandas as pd

# Replace 'file_path.parquet' with the path to your Parquet file
df = pd.read_csv('house-rent.csv')

# Display the first few rows of the dataframe
df.head(n=10)

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
5,2022-04-29,2,7000,600,Ground out of 1,Super Area,Thakurpukur,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
6,2022-06-21,2,10000,700,Ground out of 4,Super Area,Malancha,Kolkata,Unfurnished,Bachelors,2,Contact Agent
7,2022-06-21,1,5000,250,1 out of 2,Super Area,Malancha,Kolkata,Unfurnished,Bachelors,1,Contact Agent
8,2022-06-07,2,26000,800,1 out of 2,Carpet Area,"Palm Avenue Kolkata, Ballygunge",Kolkata,Unfurnished,Bachelors,2,Contact Agent
9,2022-06-20,2,10000,1000,1 out of 3,Carpet Area,Natunhat,Kolkata,Semi-Furnished,Bachelors/Family,2,Contact Owner


In [ ]:
df['RENT_PER_SQFT'] = df['Rent']/df['Size']
df

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,RENT_PER_SQFT
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,9.090909
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,25.000000
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,17.000000
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,12.500000
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner,8.823529
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner,15.000000
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner,14.500000
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent,20.000000
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent,30.000000


In [ ]:
hyd_data = df[df['City']=='Hyderabad']['RENT_PER_SQFT']

3878    19.736842
3879    55.555556
3880     9.090909
3881    18.750000
3882    18.547141
          ...    
4741    15.000000
4742    14.500000
4743    20.000000
4744    30.000000
4745    15.000000
Name: RENT_PER_SQFT, Length: 868, dtype: float64

In [ ]:
bnglr_data = df[df['City']=='Bangalore']['RENT_PER_SQFT']
bnglr_data

1496    20.870602
1497    13.125000
1498    13.000000
1499    16.346154
1500    14.967926
          ...    
2377    16.000000
2378     9.166667
2379    10.666667
2380    18.571429
2381    30.000000
Name: RENT_PER_SQFT, Length: 886, dtype: float64

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
# Assuming the dataset is named 'house_rent.csv' and it's extracted from the zip file
# df = pd.read_csv('house_rent.csv')

# Filter the data for Semi-Furnished houses in Hyderabad
filtered_df = df[(df['Furnishing Status'] == 'Semi-Furnished') & (df['City'] == 'Hyderabad')]

# Extract the relevant columns
X = filtered_df[['Size']].values  # Size in square feet
y = filtered_df['Rent'].values  # Rent in Rs

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Get the slope (coefficient of Size)
slope = model.coef_[0]
slope_per_sft = slope  # Rent per square foot

print(f'The regression slope of rent vs size for Semi-Furnished houses in Hyderabad is {slope_per_sft:.2f} Rs/sft.')

The regression slope of rent vs size for Semi-Furnished houses in Hyderabad is 32.68 Rs/sft.


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
# Assuming the dataset is named 'house_rent.csv' and it's extracted from the zip file
# df = pd.read_csv('house_rent.csv')

# Filter the data for Semi-Furnished houses in Hyderabad
filtered_df = df[(df['Furnishing Status'] == 'Semi-Furnished') & (df['City'] == 'Bangalore')]

# Extract the relevant columns
X = filtered_df[['Size']].values  # Size in square feet
y = filtered_df['Rent'].values  # Rent in Rs

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Get the slope (coefficient of Size)
slope = model.coef_[0]
slope_per_sft = slope  # Rent per square foot

print(f'The regression slope of rent vs size for Semi-Furnished houses in Bangalore is {slope_per_sft:.2f} Rs/sft.')

The regression slope of rent vs size for Semi-Furnished houses in Bangalore is 55.45 Rs/sft.


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
# Assuming the dataset is named 'house_rent.csv' and it's extracted from the zip file
# df = pd.read_csv('house_rent.csv')

# Filter the data for Semi-Furnished houses in Hyderabad
filtered_df = df[(df['Furnishing Status'] == 'Unfurnished') & (df['City'] == 'Kolkata')]

# Extract the relevant columns
X = filtered_df[['Size']].values  # Size in square feet
y = filtered_df['Rent'].values  # Rent in Rs

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Get the slope (coefficient of Size)
slope = model.coef_[0]
unfur_slope = slope  # Rent per square foot

print(f'The regression slope of rent vs size for Unfurnished houses in Kolkata is {unfur_slope:.2f} Rs/sft.')

The regression slope of rent vs size for Unfurnished houses in Kolkata is 15.69 Rs/sft.


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the dataset
# Assuming the dataset is named 'house_rent.csv' and it's extracted from the zip file
# df = pd.read_csv('house_rent.csv')

# Filter the data for Semi-Furnished houses in Hyderabad
filtered_df = df[(df['Furnishing Status'] == 'Unfurnished') & (df['City'] == 'Chennai')]

# Extract the relevant columns
X = filtered_df[['Size']].values  # Size in square feet
y = filtered_df['Rent'].values  # Rent in Rs

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Get the slope (coefficient of Size)
slope = model.coef_[0]
ch_unfur_slope = slope  # Rent per square foot

print(f'The regression slope of rent vs size for Unfurnished houses in Chennai is {ch_unfur_slope:.2f} Rs/sft.')

The regression slope of rent vs size for Unfurnished houses in Chennai is 22.07 Rs/sft.


In [ ]:
ch_unfur_slope - unfur_slope

6.386216040838537

In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

Saving est_hourly.parquet to est_hourly.parquet


In [ ]:
import pandas as pd

# Load the Parquet file
parquet_file_path = 'est_hourly.parquet'
df = pd.read_parquet(parquet_file_path)


The Parquet file has been converted to energy_consumption.csv and is ready to be opened in Excel.


In [ ]:
filtered_df = df.loc['2011-09']
filtered_df

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
Datetime,,,,,,,,,,,,
2011-09-30 01:00:00,12643.0,9380.0,1489.0,NaN,7973.0,1367.0,NaN,6238.0,NaN,24844.0,4181.0,NaN
2011-09-30 02:00:00,12046.0,8846.0,1409.0,NaN,7564.0,1332.0,NaN,5992.0,NaN,23527.0,3992.0,NaN
2011-09-30 03:00:00,11857.0,8532.0,1379.0,NaN,7316.0,1311.0,NaN,5868.0,NaN,22818.0,3898.0,NaN
2011-09-30 04:00:00,11720.0,8318.0,1374.0,NaN,7188.0,1273.0,NaN,5769.0,NaN,22530.0,3866.0,NaN
2011-09-30 05:00:00,11861.0,8279.0,1392.0,NaN,7335.0,1296.0,NaN,5888.0,NaN,22870.0,3902.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2011-09-01 21:00:00,20972.0,20666.0,3146.0,NaN,12475.0,2193.0,NaN,11260.0,NaN,37962.0,6508.0,NaN
2011-09-01 22:00:00,20357.0,20436.0,3041.0,NaN,11808.0,2101.0,NaN,10954.0,NaN,35972.0,6234.0,NaN
2011-09-01 23:00:00,18781.0,19448.0,2790.0,NaN,10646.0,1969.0,NaN,9985.0,NaN,33077.0,5744.0,NaN


In [ ]:
filtered_df = df.loc['2016-07']
filtered_df

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
Datetime,,,,,,,,,,,,
2016-07-31 01:00:00,13493.0,10985.0,1870.0,3249.0,10419.0,1566.0,1305.0,7084.0,NaN,30733.0,4726.0,NaN
2016-07-31 02:00:00,12678.0,10278.0,1742.0,3065.0,9883.0,1478.0,1242.0,6682.0,NaN,29229.0,4485.0,NaN
2016-07-31 03:00:00,12245.0,9759.0,1648.0,2899.0,9415.0,1421.0,1103.0,6352.0,NaN,28171.0,4333.0,NaN
2016-07-31 04:00:00,11870.0,9325.0,1590.0,2798.0,9258.0,1383.0,1091.0,6239.0,NaN,27537.0,4215.0,NaN
2016-07-31 05:00:00,11637.0,9094.0,1551.0,2712.0,9163.0,1364.0,1071.0,6208.0,NaN,27253.0,4176.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-07-01 21:00:00,15351.0,11667.0,2144.0,3617.0,13496.0,1728.0,1695.0,7754.0,NaN,37981.0,5852.0,NaN
2016-07-01 22:00:00,14813.0,11290.0,2069.0,3458.0,13007.0,1687.0,1645.0,7605.0,NaN,37018.0,5716.0,NaN
2016-07-01 23:00:00,14059.0,11147.0,1984.0,3210.0,12053.0,1571.0,1548.0,7202.0,NaN,34442.0,5307.0,NaN


In [ ]:
# Save the DataFrame as a CSV file
csv_file_path = 'july_2016.csv'
filtered_df.to_csv(csv_file_path, index=True)

print(f'The Parquet file has been converted to {csv_file_path} and is ready to be opened in Excel.')

The Parquet file has been converted to july_2016.csv and is ready to be opened in Excel.


In [ ]:
df.tail()

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
Datetime,,,,,,,,,,,,
2018-01-01 20:00:00,21089.0,13858.0,2732.0,4426.0,18418.0,1962.0,2866.0,9378.0,NaN,44284.0,8401.0,NaN
2018-01-01 21:00:00,20999.0,13758.0,2724.0,4419.0,18567.0,1940.0,2846.0,9255.0,NaN,43751.0,8373.0,NaN
2018-01-01 22:00:00,20820.0,13627.0,2664.0,4355.0,18307.0,1891.0,2883.0,9044.0,NaN,42402.0,8238.0,NaN
2018-01-01 23:00:00,20415.0,13336.0,2614.0,4224.0,17814.0,1820.0,2880.0,8676.0,NaN,40164.0,7958.0,NaN
2018-01-02 00:00:00,19993.0,12816.0,2552.0,4100.0,17428.0,1721.0,2846.0,8393.0,NaN,38608.0,7691.0,NaN


In [ ]:
!pip install pandas sqlalchemy pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
hostname = 'db.relational-data.org'
port = 3306
username = 'guest'
password = 'relational'
database = 'restbase'

# Create a connection to the database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}:{port}/{database}')

In [ ]:
# List all tables in the database
tables = pd.read_sql('SHOW TABLES', engine)

In [ ]:
tables

,Tables_in_restbase
0,generalinfo
1,geographic
2,location


In [ ]:
# List all tables in the database
def query(q):
  return pd.read_sql(q, engine)

In [ ]:
city_bakery = query(
    """
    SELECT
        city, COUNT(*) as count_bakery
    FROM
        generalinfo
    where food_type = 'bakery'
    group by
        city
    """
)
city_bakery

,city,count_bakery
0,antioch,2
1,aptos,1
2,berkeley,1
3,bethel island,1
4,burlingame,2
5,calistoga,1
6,capitola,1
7,daly city,1
8,el cerrito,1
9,fairfield,1


In [ ]:
city_food = query(
    """
    SELECT
        city, COUNT(*) as count_city
    FROM
        generalinfo
    group by
        city
    """
)

In [ ]:
city_food

,city,count_city
0,alameda,130
1,alamo,12
2,albany,38
3,alviso,3
4,american canyon,3
...,...,...
162,watsonville,51
163,west pittsburg,3
164,winters,1
165,woodside,10


In [ ]:
merged_df = pd.merge(city_food, city_bakery, on='city')
merged_df

,city,count_city,count_bakery
0,antioch,81,2
1,aptos,35,1
2,berkeley,327,1
3,bethel island,2,1
4,burlingame,96,2
5,calistoga,34,1
6,capitola,40,1
7,daly city,73,1
8,el cerrito,48,1
9,fairfield,100,1


In [ ]:
merged_df['percentage_bakery'] = merged_df['count_bakery'] / merged_df['count_city'] * 100
merged_df

,city,count_city,count_bakery,percentage_bakery
0,antioch,81,2,2.469136
1,aptos,35,1,2.857143
2,berkeley,327,1,0.305810
3,bethel island,2,1,50.000000
4,burlingame,96,2,2.083333
5,calistoga,34,1,2.941176
6,capitola,40,1,2.500000
7,daly city,73,1,1.369863
8,el cerrito,48,1,2.083333
9,fairfield,100,1,1.000000


In [ ]:
sanjose_perc = merged_df[merged_df['city'] == 'san jose']['percentage_bakery']
sanjose_perc.values[0]

0.4287245444801715

In [ ]:
daly_city_perc = merged_df[merged_df['city'] == 'daly city']['percentage_bakery']
daly_city_perc.values[0]

1.36986301369863

In [ ]:
merged_df[merged_df['percentage_bakery'] > sanjose_perc.values[0]].count()

city                 30
count_city           30
count_bakery         30
percentage_bakery    30
dtype: int64

In [ ]:
merged_df[merged_df['percentage_bakery'] > daly_city_perc.values[0]].count()

city                 17
count_city           17
count_bakery         17
percentage_bakery    17
dtype: int64